In [150]:
###10분짜리 6초윈도우 슬라이드하면서
import pickle
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import svm
from datetime import datetime
import glob


filename_queue=glob.glob('./rawdata/compdata[0-9]*.csv')
#feature extration from data
def feature_extraction(window):
    mean=window.mean(axis=-1)
    stddev=window.std(axis=-1)
    median=np.median(window,axis=-1)
    percent25=np.percentile(window,25,axis=-1)
    percent75=np.percentile(window,75,axis=-1)
    return(np.array([mean,stddev,median,percent25,percent75]))

#data[0:600] 각각 1sec
#windowSize=6sec=6
#stepSize=1sec=1
#len(data)=600=600sec=10minute
def sliding_window(data,stepSize,windowSize):
    for x in range(0,len(data)-windowSize+1,stepSize):
        yield(x,data[x:x+windowSize])

def pred_window(clf,data):
    test_set=[]
    for(x,window) in sliding_window(data,1,6): 
        wind=window.reshape(180)
        test_set.append(wind)
    print(np.shape(test_set))
    return clf.predict(test_set)

In [151]:
#showing shape of the data
for file in filename_queue:
    df=pd.read_csv(file)
    print(file,df.shape)



./rawdata/compdata7.csv (20, 18003)
./rawdata/compdata6.csv (19, 18003)
./rawdata/compdata10.csv (19, 18003)
./rawdata/compdata9.csv (19, 18003)
./rawdata/compdata5.csv (20, 18003)
./rawdata/compdata8.csv (39, 18003)


In [152]:
for k in range(4):
    print(k)

0
1
2
3


In [153]:
return_path="./featuredata/compfeature2.csv"
file=open(return_path,'a')
writer=csv.writer(file)
#writer.writerows([["context"],["data"]])  
#showing shape of the data
for filename in filename_queue:
    file=open(filename, newline='')
    index=0    

    for j in range(2): 
        ndata=[]
        file.seek(0)
        reader=csv.reader(file)
        linecount=0
        for row in reader:
            row[3:]=[float(i) for i in row[3:]]
            linecount+=1
            if(linecount>20):
                continue
            if(int(row[2])==0):
                time=int(row[0]) #minutewise
                context=[row[1]] #drink 0, eat 1, cafe 2, desk 3
                sensor=int(row[2]) #acc:0 gyro:1
                X_acc=[row[4*i+3+2*j] for i in range(1500)] #1500 =60sec/40ms
                Y_acc=[row[4*i+6003+2*j] for i in range(1500)]
                Z_acc=[row[4*i+12003+2*j] for i in range(1500)]
            if(int(row[2])==1):
                X_gyro=[row[4*i+3+2*j] for i in range(1500)]
                Y_gyro=[row[4*i+6003+2*j] for i in range(1500)]
                Z_gyro=[row[4*i+12003+2*j] for i in range(1500)]
                list=[X_acc,Y_acc,Z_acc,X_gyro,Y_gyro,Z_gyro]
                
                for x in range(0,60):
                    wind=np.array([X_acc[25*x:25*x+25],Y_acc[25*x:25*x+25],Z_acc[25*x:25*x+25],
                                X_gyro[25*x:25*x+25],Y_gyro[25*x:25*x+25],Z_gyro[25*x:25*x+25]])  
                    feature=feature_extraction(wind)
                    feature=feature.reshape(6,5)
                    ndata.append(feature)
                
        ndata=np.array(ndata)  #600, 6, 5 # format to be predicted
        flatten_data=ndata.flatten() #18000, format to be saved
        print(len(flatten_data))
        row_data=flatten_data
        writer.writerows([context,row_data])



18000
18000
18000
18000
18000
18000
18000
18000
18000
18000
18000
18000
